In [10]:
# Importing packages
import pandas as pd
import pandas as pd
import requests
import csv
import json
import time
import os

In [14]:
# Specify the directory path for saving files
save_dir = "../2. analysis"

# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [2]:
def fetch_results(url):
    results = []
    page_token = None

    while True:
        response = requests.get(url, params={'pagetoken': page_token})
        data = response.json()

        # Append the current page results to the main results list
        results.extend(data['results'])

        # Check if there are more pages available
        if 'next_page_token' in data:
            page_token = data['next_page_token']
            # Wait for a short duration to ensure the next page token is valid
            time.sleep(2)
        else:
            break

    return results

In [3]:
state_dict = {"California": ['Los Angeles', 'San Diego', 'San Jose','San Fransisco', 'Fresno','Sacremento','Long Beach','Oakland'],\
     "Texas": ['Houston', 'San Antonio', 'Dallas']}

# Note that Google doesn't allow a private API key to be uploaded on GitHub, so this key is usually shared\
# through private channels; namely email/Slack. Even better if each individual has their own key, not\
# mandatory though
api_key = "#Ask_for_this"
keyword = "custom builders in"

In [15]:
def get_builder_csv(state_dict):
    # Function will take a state (and its cities) as input and return list of 50+ custom builders per city. 
    # Results across cities will be appended and one csv for state will be generated
      
    # Input params
    # state_dict: 

    #Output: csv with list of builders for the state
    for state, cities in state_dict.items():
        state_builders = []

        for city in cities:
            location = city + ', ' + state
            url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query={keyword}+{location}&key={api_key}'
            city_builders = fetch_results(url)  # Fetch builders data for the city
            # Save the city builders data as JSON
            city_json_filename = os.path.join(save_dir, f'{state}_{city}_builders.json')
            with open(city_json_filename, 'w') as file:
                json.dump(city_builders, file)

            state_builders.extend(city_builders)  # Append city builders to the state builders list

        # Convert the state builders list to a dataframe
        state_dataframe = pd.DataFrame(state_builders)

        # Save the state dataframe as CSV
        state_csv_filename = os.path.join(save_dir, f'{state}_builders.csv')
        state_dataframe.to_csv(state_csv_filename, index=False)

        print(f'CSV file created for state: {state}')

In [16]:
state_dict = {"California": ['Los Angeles', 'San Diego', 'San Jose','San Fransisco', 'Fresno','Sacremento','Long Beach','Oakland']}
get_builder_csv(state_dict)

CSV file created for state: California
